In [88]:
#dependencies
import pandas as pd
import os


In [128]:
json_path = os.path.join('purchase_data2.json')
file = pd.read_json(json_path)

In [139]:
#Player Count
#* Total Number of Players

player_counts = file["SN"].nunique()
player_df = pd.DataFrame([player_counts])
player_df

,0
0,74


In [150]:
# **Purchasing Analysis (Total)**

# * Number of Unique Items
unique_items = file["Item Name"].nunique()

# * Average Purchase Price
#avgprice = file["Price"].groupby(file["Item Name"]).mean()
avgprice = '${:,.2f}'.format(file["Price"].mean())

# * Total Number of Purchases
purchases = file[file["Price"] > 0].count()["Price"]

# * Total Revenue
total_rev = '${:,.2f}'.format(file["Price"].sum())
#total_rev_format = '${:,.2f}'.format(total_rev)

# print('There are',str(unique_items), 'unique items.')
# print('The average price per item is',str(avgprice) + '.')
# print('The total revenue is', str(total_rev)+'.')
# total_rev_format

pa = {'#ofUnq': [unique_items], 'Avgprice': [avgprice], 'total#purchase': [purchases],'total_rev': [total_rev]  }

purchaseA = pd.DataFrame(pa)
purchaseA

,#ofUnq,Avgprice,total#purchase,total_rev
0,63,$2.92,78,$228.10


In [151]:
#SexDF delete duplicates
test_df = file[["Gender", "SN"]]
nodup_test_df = test_df.drop_duplicates()

#Population total
Total = nodup_test_df["Gender"].count()
# * Percentage and Count of Male Players
Males = nodup_test_df[nodup_test_df["Gender"] == 'Male'].count()["Gender"]
Percentage_Male = Males / Total
pmale = '{:,.2f}%'.format(Percentage_Male * 100)

# * Percentage and Count of Female Players
Females = nodup_test_df[nodup_test_df["Gender"] == 'Female'].count()["Gender"]
Percentage_Female = Females / Total
pfemale = '{:,.2f}%'.format(Percentage_Female * 100)

# * Percentage and Count of Other / Non-Disclosed
Neither = Total - Males - Females
Percentage_other = Neither / Total
pother = '{:,.2f}%'.format(Percentage_other * 100)

# nodup_test_df = test_df.drop_duplicates()
# group_test = nodup_test_df.groupby(['Gender']).count()


ga = {'Gender': ['Males', 'Females', 'Neither'], 'Percentage of players': [pmale, pfemale, pother], 'Total_Count': [Males, Females, Neither]}

gendA = pd.DataFrame(ga)
gendA



,Gender,Percentage of players,Total_Count
0,Males,81.08%,60
1,Females,17.57%,13
2,Neither,1.35%,1


In [156]:

# * The below each broken by gender
#   * Purchase Count
Gender_df = file[["Gender", "Price"]]
gender_count = Gender_df.groupby(['Gender']).count
Total_C = Gender_df["Price"].count()
Male_C = (Gender_df[Gender_df["Gender"] == 'Male'].count()["Price"])
Female_C = (Gender_df[Gender_df["Gender"] == 'Female'].count()["Price"])
Other_C = Total_C - Male_C - Female_C

#   * Average Purchase Price
avgprice_g = Gender_df["Price"].groupby(Gender_df["Gender"]).mean()
Total_C = Gender_df["Price"].count()
Male_A = (Gender_df[Gender_df["Gender"] == 'Male'].mean()["Price"])
Female_A = (Gender_df[Gender_df["Gender"] == 'Female'].mean()["Price"])
Other_A = (Gender_df[~Gender_df["Gender"].isin(['Male', 'Female'])]).mean()["Price"]
Male_AF = '${:,.2f}'.format(Male_A)
Female_AF = '${:,.2f}'.format(Female_A)
Other_AF = '${:,.2f}'.format(Other_A)

#   * Total Purchase Value
# sumprice_g = Gender_df["Price"].groupby(Gender_df["Gender"]).sum()

Male_S = (Gender_df[Gender_df["Gender"] == 'Male'].sum()["Price"])
Female_S = (Gender_df[Gender_df["Gender"] == 'Female'].sum()["Price"])
Other_S = (Gender_df[~Gender_df["Gender"].isin(['Male', 'Female'])]).sum()["Price"]
#   * Normalized Totals (normalizing for the # of people in each gender group)
Male_P = (Gender_df[Gender_df["Gender"] == 'Male'].sum()["Price"])/ Males
Female_P = (Gender_df[Gender_df["Gender"] == 'Female'].sum()["Price"])/Females
Other_P = (Gender_df[~Gender_df["Gender"].isin(['Male', 'Female'])]).sum()["Price"]/Neither

Male_PF = '${:,.2f}'.format(Male_P)
Female_PF = '${:,.2f}'.format(Female_P)
Other_PF = '${:,.2f}'.format(Other_P)

gga = {'1. Gender': ['Males', 'Females', 'Undisclosed/Other'],
      '2. Purchase Count': [Male_C, Female_C, Other_C], '3. Avg Purchase Price': [Male_AF, Female_AF, Other_AF],
       '4. Purchase Sum': [Male_S, Female_S, Other_S],'5. Avg Purchase Norm': [Male_PF, Female_PF, Other_PF],}

ggendA = pd.DataFrame(gga)
ggendA




,1. Gender,2. Purchase Count,3. Avg Purchase Price,4. Purchase Sum,5. Avg Purchase Norm
0,Males,64,$2.88,184.60,$3.08
1,Females,13,$3.18,41.38,$3.18
2,Undisclosed/Other,1,$2.12,2.12,$2.12


In [190]:
# * The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 
bins = [0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40]
group_names = ['a', 'b', 'c', 'd','e','f','h', 'i', 'j', 'k']
Age_df = file[["Age", "Price", "SN"]]


pd.cut(Age_df["Age"], bins, labels=group_names)
Age_df["Age_Group"] = pd.cut(Age_df["Age"],
                                           bins, labels=group_names)

Age_df2 = Age_df.drop_duplicates(subset = ['SN'])
Age_nom = Age_df2['Age_Group'].groupby(Age_df2['Age_Group']).count()

#number of real players per age group

# #   * Purchase Count
age_count = Age_df['Price'].groupby(Age_df['Age_Group']).count()

# #   * Average Purchase Price
age_mean = Age_df['Price'].groupby(Age_df['Age_Group']).mean()
# #   * Total Purchase Value
age_sum = Age_df['Price'].groupby(Age_df['Age_Group']).sum()
# #   * Normalized Totals (normalizing for the # of people in each age group)
age_norm = age_sum / Age_nom

age_pr = (Age_nom / Age_nom.sum())



dfag = pd.concat([Age_nom,age_pr, age_mean, age_sum, age_norm], axis=1)

dfag.columns = ['Count','Percentage', 'Avg purchase price', 'Total purchase price', 'Normalized']

dfag.Percentage = dfag.Percentage.astype(float).fillna(0.0) * 100
dfag['Percentage'] = dfag['Percentage'].map('{:,.2f}%'.format)
dfag['Avg purchase price'] = dfag['Avg purchase price'].astype(float).fillna(0.0)
dfag['Total purchase price'] = dfag['Total purchase price'].astype(float).fillna(0.0)
dfag['Normalized'] = dfag['Normalized'].astype(float).fillna(0.0)
dfag['Avg purchase price'] = dfag['Avg purchase price'].map('${:,.2f}'.format)
dfag['Total purchase price'] = dfag['Total purchase price'].map('${:,.2f}'.format)
dfag['Normalized'] = dfag['Normalized'].map('${:,.2f}'.format)

dfag



# print(age_count)
# print(age_mean)
# print(age_sum)
# print(age_sum / Age_nom)

C:\Users\HeeXPS\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Count,Percentage,Avg purchase price,Total purchase price,Normalized
Age_Group,,,,,
a,0,0.00%,$0.00,$0.00,$0.00
b,4,5.41%,$2.32,$9.29,$2.32
c,4,5.41%,$3.37,$13.49,$3.37
d,4,5.41%,$2.87,$11.49,$2.87
e,17,22.97%,$2.73,$46.45,$2.73
f,24,32.43%,$3.13,$81.36,$3.39
h,8,10.81%,$2.90,$26.11,$3.26
i,3,4.05%,$1.99,$7.97,$2.66
j,7,9.46%,$2.93,$20.48,$2.93


In [192]:
# * Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
#   * SN
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value
topspend = file.groupby(["SN"])["Price"].agg("sum")
topspendc = file.groupby(["SN"])["Price"].agg("count")
topspendm = file.groupby(["SN"])["Price"].agg("mean")
# topspendsort = topspend.iloc[np.lexsort([topspend.values], order = "desc")]
# topspendsort
tdf = pd.concat([topspend, topspendc, topspendm], axis=1)
# data.iloc[:,0]


tdf.columns = ['Total','Count', 'Avg']
tdf['Total'] = tdf['Total'].map('${:,.2f}'.format)
tdf['Avg'] = tdf['Avg'].map('${:,.2f}'.format)

tdf.sort_values(by=['Total'], ascending=False).head()


,Total,Count,Avg
SN,,,
Sundaky74,$7.41,2,$3.71
Aidaira26,$5.13,2,$2.56
Eusty71,$4.81,1,$4.81
Chanirra64,$4.78,1,$4.78
Alarap40,$4.71,1,$4.71


In [193]:
# * Identify the 5 most popular items by purchase count, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value
itopspend = file.groupby(["Item Name"])["Price"].agg("sum")
itopspendc = file.groupby(["Item Name"])["Price"].agg("count")
itopspendm = file.groupby(["Item Name"])["Price"].agg("mean")
# topspendsort = topspend.iloc[np.lexsort([topspend.values], order = "desc")]
# topspendsort
itdf = pd.concat([itopspend, itopspendc, itopspendm], axis=1)
# data.iloc[:,0]

itdf.columns = ['Total','Count', 'Price']
itdf['Total'] = itdf['Total'].map('${:,.2f}'.format)
itdf['Price'] = itdf['Price'].map('${:,.2f}'.format)
itdf.sort_values(by=['Count'], ascending=False).head()

,Total,Count,Price
Item Name,,,
Mourning Blade,$10.92,3,$3.64
"Deadline, Voice Of Subtlety",$2.58,2,$1.29
Stormcaller,$5.54,2,$2.77
Relentless Iron Skewer,$4.24,2,$2.12
Apocalyptic Battlescythe,$8.98,2,$4.49


In [194]:
# * Identify the 5 most profitable items by total purchase value, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value
itdf.sort_values(by=['Total'], ascending=False).head()

,Total,Count,Price
Item Name,,,
"Heartstriker, Legacy of the Light",$9.42,2,$4.71
Apocalyptic Battlescythe,$8.98,2,$4.49
Betrayer,$8.24,2,$4.12
Feral Katana,$8.22,2,$4.11
Stormcaller,$5.54,2,$2.77


In [2]:
#observations

#1. There seems to be no coorelation between age and the $ spent, little worrysome for parents.

#2. Although the sample size is small, the normalized average purchase being higher for females is interesting. Since its a F2P game, this may hint that women are less likely than men to play for items

#3. It is concerning that the top spenders are ones who purchase either 1 or 2 items, doesnt seem to have too much repeatability.